## Alcholic Anonymous News articles extraction from India Today. Data Extraction of following parameters
- Headline
- Description
- Author
- Published Date
- Category
- Publication
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions

### Creating Empty lists for Alcholic Anonymous News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls, category, publication = [], [], [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results¶

In [3]:
keyword = 'Alcoholics Anonymous site:www.indiatoday.in'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

soup = BeautifulSoup(get(url).text, 'lxml')
try:
    # Extracts the digits if it the resulted number without comma ','. eg: About 680 results (0.23 seconds)
    max_pages = round([int(s) for s in soup.select_one('div#resultStats').text.split() if s.isdigit()][0]/10)
    max_pages = max_pages + 1
except:
    # Extracts the digits if it the resulted number without comma ','. eg: About 1,080 results (0.23 seconds)
    max_pages = round(int(''.join(i for i in soup.select_one('div#resultStats').text if i.isdigit()))/10)
    max_pages = max_pages + 1

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [5]:
urls = list(dict.fromkeys(urls))
print("Total Extracted URL's are" + ' : ' + str(len(urls)), type(urls))

Total Extracted URL's are : 57 <class 'list'>


### Iterates urls through for loop. Scraping the Articles with above parameters

In [6]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            try:
                headlines.append(article.title.strip())
            except:
                headlines.append(soup.select_one('meta[property="og:title"]')['content'].strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            try:
                descriptions.append(soup.select_one('meta[property="og:description"]')['content'].strip().replace('\n', ' '))
            except:
                descriptions.append(article.meta_description.strip())
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            try:
                authors.append(soup.select_one('meta[name="author"]')['content'].strip())
            except:
                authors.append(article.authors.strip())
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            try:
                dates.append(soup.select_one('meta[property="article:published_time"]')['content'].strip())
            except:
                dates.append(str(article.publish_date))
        except:
            dates.append(None)
            
        # Extracts the news articles
        try:
            try:
                news.append(' '.join(soup.select_one('.storyText').text.split()).replace('\n', '').strip())
            except:
                news.append(' '.join(article.text.split()).replace("\'\'"," ").replace("\'", "").replace(" / ", ""))
        except:
            news.append(None)
            
        # Extracts the news category
        try:
            category.append(article.meta_data['category'])
        except:
            category.append(None)
            
        # Extracts the news publication
        try:
            publication.append(article.meta_data['og']['site_name'])
        except:
            publication.append(None)

        # Extracts Keywords and Summaries
        try:
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
                        
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        category.append(None)
        publication.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 3min 10sdiatoday.in/movies/hollywood/story/viral-videos-stoke-celebrities-images-156049-2013-03-13ecret-sessions-sonal-holland-wine-club-royal-opera-house-733710-2016-10-21


### Checking Array Length of each list to create DataFrame

In [7]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(category), len(publication), len(news), len(keywords), len(summaries), len(urls))

57 57 57 57 57 57 57 57 57 57


### Creating a csv file after checking array length and droping the missing values from the dataset

In [8]:
if len(headlines) == len(descriptions) == len(authors) == len(dates) == len(news) == len(publication) == len(keywords) == len(summaries) == len(urls) == len(category):
    tbl = pd.DataFrame({'Headlines' : headlines,
                        'Descriptions' : descriptions,
                        'Authors' : authors,
                        'Published_Dates' : dates,
                        'Publication' : publication,
                        'Articles' : news,
                        'category' : category,
                        'Keywords' : keywords,
                        'Summaries' : summaries,
                        'Source_URLs' : urls})
    tbl.dropna()
    path = 'C:\\Users\\GM\\Desktop\\!Code!\\CDRI\\Alcoholics Anonymous\\Data Extraction\\#Dataset\\'
    tbl.to_csv(path+'India_Today.csv', index=False)
else:
    print('Array lenght does not match!')

tbl.head()

Headlines  \
0  One of the most successful ways of treating ad...   
1  Original 1939 Alcoholics Anonymous Manuscript ...   
2  Demi Lovato left Met Gala to go to Alcoholics ...   
3  Compounded by denial and loneliness, alcoholis...   
4                                 Alcohol withdrawal   

                                        Descriptions Authors  \
0  Today the T.T. Ranganathan Clinical Research C...    None   
1  The original manuscript of Alcoholics Anonymou...    None   
2  Demi Lovato left Met Gala to go to Alcoholics ...    None   
3  Compounded by denial and a dearth of treatment...    None   
4  150,2009,2011,24th,373,492,501,911,abnormal,ab...    None   

             Published_Dates  Publication  \
0  2014-01-27T16:19:47+05:30  India Today   
1        2017-05-19 00:00:00  India Today   
2  2018-03-10T11:05:11+05:30  India Today   
3  2013-06-20T14:12:29+05:30  India Today   
4  2014-09-24T09:54:54+05:30  India Today   

                                            Articles category  \
0  Aversion therapy: effective Aversion therapy: ...       {}   
1  The original manuscript of Alcoholics Anonymou...       {}   
2  Los Angeles, Mar 10 (PTI) Demi Lovato has reca...       {}   
3  More than anything else, this letter from her ...       {}   
4  Definition Alcohol withdrawal refers to sympto...       {}   

                                            Keywords  \
0  [ways, addicts, aa, anonymous, alcoholics, pat...   
1  [wilson, million, york, anonymous, alcoholics,...   
2  [met, pti, meeting, anonymous, alcoholics, dia...   
3  [women, loneliness, problem, drinking, alcohol...   
4  [drinking, withdrawal, medicine, stop, treatme...   

                                           Summaries  \
0  It first admits patients to a nursing home for...   
1  The original manuscript of Alcoholics Anonymou...   
2  Los Angeles, Mar 10 (PTI) Demi Lovato has reca...   
3  Now, there are 30 to 35 every month and the in...   
4  DefinitionAlcohol withdrawal refers to symptom...   

                                         Source_URLs  
0  https://www.indiatoday.in/magazine/cover-story...  
1  https://www.indiatoday.in/world-special/video/...  
2  https://www.indiatoday.in/pti-feed/story/demi-...  
3  https://www.indiatoday.in/magazine/living/stor...  
4  https://www.indiatoday.in/lifestyle/health/sto...

In [9]:
tbl.shape

(57, 10)